In [10]:
import anndata as ad
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, precision_recall_curve, auc, precision_score, recall_score
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn

from src.perturbation_module.baseline_model_concat.model import ConditionalFeedForwardNN
from src.perturbation_module.baseline_model_concat.dataset_zhao import ZhaoDatasetBaseline
import yaml
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
import torch.nn as nn

In [2]:
ROOT = 'C:\\Users\\curea\\Documents\\bioFM for drug discovery\\dege-fm\\'

adata_zhao = ad.read_h5ad(ROOT + "data\\zhao\\zhao_preprocessed.h5ad")

In [3]:
with open(ROOT+ "config\\baseline.yaml", 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ConditionalFeedForwardNN(config)

model.load_state_dict(torch.load(ROOT + "results\\baseline\\model_weights.pth"))


C:\Users\curea\AppData\Local\Temp\ipykernel_19684\2046137749.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(ROOT + "results\\baseline\

<All keys matched successfully>

In [4]:
def test(model, dataloader, criterion, device):
    model.eval()
    test_losses = list()
    res = list()
    model_meta = model.get_meta()

    with torch.no_grad():
        for inputs, targets, meta in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            res.append({"input": inputs, "targets": targets, "predicted": outputs, "meta": meta})

            loss = criterion(outputs, targets)

            test_losses.append(loss.item())

        avg_loss = np.mean(test_losses)
        print(f"Test Loss: {avg_loss}")
    return res

In [5]:
zhao_dataset = ZhaoDatasetBaseline(config['dataset_params']['zhao_adata_path'])

 50%|█████     | 5/10 [07:08<10:16, 123.37s/it]

PW031 has no perturbations, skipping ...


100%|██████████| 10/10 [14:23<00:00, 86.39s/it] 


NameError: name 'DataLoader' is not defined

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

zhao_loader = DataLoader(zhao_dataset, batch_size=config['train_params']['batch_size'], shuffle=True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
results_zhao = test(model, zhao_loader, criterion, device)

Test Loss: 0.0009011158362716005


In [14]:
import pickle as pkl

with open(ROOT + "results\\baseline\\zhao_predictions.pkl", "wb") as f:
    pkl.dump(results_zhao, f)